In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd

In [2]:
# 1. MARS NEWS SITE

# URL of page to be scraped -- we're going to SCRAP the WEB PAGE. Instead of using a path to a File.
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)


In [3]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())

In [4]:
# first Title
news_title = soup.find("div",class_="content_title").text
print(news_title)



Mars Helicopter Attached to NASA's Perseverance Rover




In [5]:
# first Parragraph text
news_p = soup.find("div", class_="rollover_description_inner").text 
print(news_p)



The team also fueled the rover's sky crane to get ready for this summer's history-making launch.



In [6]:
# 2. JPL MARS SPACE IMAGE

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [7]:
#Click on button "FULL IMAGE"
browser.find_by_id('full_image').first.click()

In [8]:
#Click on button "MORE INFO"
browser.find_link_by_partial_text('more info').first.click()

In [9]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve full size image partial link (JPG format)
feature_image = soup.find("figure", class_="lede").a["href"]
print(feature_image)

/spaceimages/images/largesize/PIA17172_hires.jpg


In [10]:
#Link of full size image - with HTTPS:// link
featured_image_url ="https://www.jpl.nasa.gov/" + feature_image
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA17172_hires.jpg


In [ ]:
#Close Browser
browser.quit()

In [11]:
# 3. MARS WEATHER TWEET

url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')


In [12]:
#Get the first Tweet Parragraph
body_content =soup.body.find("p", class_="tweet-text")

tweets=[]
for body in body_content:
    tweets.append(body)

mars_weather=tweets[0]
print(mars_weather)


InSight sol 485 (2020-04-07) low -93.4ºC (-136.1ºF) high -7.0ºC (19.4ºF)
winds from the WNW at 5.2 m/s (11.5 mph) gusting to 17.4 m/s (38.9 mph)
pressure at 6.50 hPa
